<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [3]:
import bokeh
import pandas as pd

In [6]:
%pwd

'C:\\Users\\91994\\Downloads\\Data-Science-Hacks-master\\Code'

In [7]:
data = pd.read_csv('../../gapminder.tsv', delimiter='\t',thousands=',',index_col='year')
data.head()

,country,continent,lifeExp,pop,gdpPercap
year,,,,,
1952,Afghanistan,Asia,28.801,8425333,779.445314
1957,Afghanistan,Asia,30.332,9240934,820.853030
1962,Afghanistan,Asia,31.997,10267083,853.100710
1967,Afghanistan,Asia,34.020,11537966,836.197138
1972,Afghanistan,Asia,36.088,13079460,739.981106


In [8]:
data = data.rename(columns={'pop':'population'});
data.head()

,country,continent,lifeExp,population,gdpPercap
year,,,,,
1952,Afghanistan,Asia,28.801,8425333,779.445314
1957,Afghanistan,Asia,30.332,9240934,820.853030
1962,Afghanistan,Asia,31.997,10267083,853.100710
1967,Afghanistan,Asia,34.020,11537966,836.197138
1972,Afghanistan,Asia,36.088,13079460,739.981106


Generating an Empty canvas with Bokeh

Let's generate a bare empty figure with Bokeh just to get things started

In [9]:
from bokeh.io import output_notebook, output_file

output_notebook()  # Render inline in a Jupyter Notebook or
#output_file('filename.html')  # Render to static HTML

Loading BokehJS ...

In [10]:
from bokeh.io import show
from bokeh.plotting import figure

# Set up a generic figure() object
fig = figure()

# See what it looks like
show(fig)

In [11]:
# Basic Plot 

fig = figure(x_range=(100,1000),
             y_range=(0,10),
             background_fill_color='pink',
             background_fill_alpha=0.2,
             plot_height=400,
             plot_width=600,
             x_axis_label='X Label',
             y_axis_label='Y Label',
             title='Example Figure',
             toolbar_location='below',
             tools='save')

# See what it looks like
show(fig)

In [12]:
# Circles

p = figure(plot_width=300, plot_height=300)

# add a circle renderer with a size, color, and alpha
p.circle([1, 2, 3, 4, 5], [6, 7, 1, 4, 3], size=20, color="green", alpha=0.5)

# show the results
show(p)

In [13]:
from bokeh.io import show
from bokeh.plotting import figure

p = figure(height = 400, width = 400)
p.circle(x = data['gdpPercap'], y = data['lifeExp'])
p.xaxis.axis_label = "GDP per Capita"
p.yaxis.axis_label = "Life Expectancy"
show(p)

In [14]:
from bokeh.io import show
from bokeh.plotting import figure


p = figure(
    height = 400, 
    width = 600,
    x_axis_type = 'log',
    x_range =(100,100000), y_range = (0,100),
    title = "Life Expectancy vs GDP of Nations ",)

p.circle(x = data['gdpPercap'], y = data['lifeExp'],color="orange",alpha = 0.6)
p.xaxis.axis_label = "GDP per Capita"
p.yaxis.axis_label = "Life Expectancy(Years)"


from bokeh.models import NumeralTickFormatter
p.xaxis[0].formatter = NumeralTickFormatter(format='$0,')


show(p)


In [15]:
# Choosing year 2007

from bokeh.models import ColumnDataSource
source = ColumnDataSource(data.loc[2007])

In [16]:
# Adding the source option in our plot

from bokeh.io import show
from bokeh.plotting import figure

p = figure(
    height = 400, 
    width = 600,
    x_axis_type = 'log',
    x_range = (100,100000), y_range = (0,100),
    title = "Life Expectancy vs GDP of Nations for 2010")

p.circle(x='gdpPercap', y='lifeExp',color="orange",alpha=0.6,source=source)
p.xaxis.axis_label = "GDP per Capita"
p.yaxis.axis_label = "Life Expectancy"


from bokeh.models import NumeralTickFormatter
p.xaxis[0].formatter = NumeralTickFormatter(format='$0,')

show(p)

In [17]:

from bokeh.io import show
from bokeh.models import LinearInterpolator
from bokeh.plotting import figure


p = figure(
    height = 400, 
    width = 600,
    x_axis_type = 'log',
    x_range = (100,100000), y_range = (0,100),
    title = "Life Expectancy vs GDP of Nations for 2010")


size_mapper = LinearInterpolator(
    x = [data.population.min(), data.population.max()],
    y = [10,100]
)


p.circle(x='gdpPercap', y='lifeExp',color="orange",alpha=0.6,source=source,
         size={'field':'population', 'transform':size_mapper})

p.xaxis.axis_label = "GDP per Capita"
p.yaxis.axis_label = "Life Expectancy"


from bokeh.models import NumeralTickFormatter
p.xaxis[0].formatter = NumeralTickFormatter(format='$0,')

show(p)

In [ ]:
from bokeh.io import show
from bokeh.models import LinearInterpolator
from bokeh.plotting import figure
from bokeh.models import CategoricalColorMapper
from bokeh.palettes import Spectral6
from bokeh.models import HoverTool


hover = HoverTool(tooltips='@country')

p = figure(
    height = 400, 
    width = 600,
    x_axis_type = 'log',
    tools = [hover],
    x_range = (100,100000), y_range = (0,100),
    title = "Life Expectancy vs GDP of Nations")


size_mapper = LinearInterpolator(
    x = [data.population.min(), data.population.max()],
    y = [10,100]
)

# Mapping color to continent
color_mapper = CategoricalColorMapper(
    factors=list(data.continent.unique()),
    palette=Spectral6
)


from bokeh.io import push_notebook
def update(year):
    new_data = dict(
        gdpPercap=data.loc[year].gdpPercap,
        lifeExp=data.loc[year].lifeExp,
        country=data.loc[year].country,
        population=data.loc[year].population,
        continent=data.loc[year].continent
    )
    source.data = new_data
    if len(p.title.text) > len("Life Expectancy vs GDP of Nations "):
        tmp = str(p.title.text)
        p.title.text=tmp[:-4]
        p.title.text = p.title.text + str(year)
    else:
        p.title.text = p.title.text + str(year)
    push_notebook()


p.circle(x='gdpPercap', y='lifeExp',alpha=0.6,source=source,
         size={'field':'population', 'transform':size_mapper},
         color={'field':'continent', 'transform':color_mapper},
         legend='continent'
        )

p.xaxis.axis_label = "GDP per Capita"
p.yaxis.axis_label = "Life Expectancy"


from bokeh.models import NumeralTickFormatter
p.xaxis[0].formatter = NumeralTickFormatter(format='$0,')

# formating the legend
p.legend.location = (30, -5)
p.right.append(p.legend[0])
p.legend.border_line_color = None


show(p,notebook_handle=True)

import time
year = 1952
while True:
    update(year)
    if year == 2007:
        year=1952
    else:
        year+=5
    time.sleep(0.5)
    continue

In [ ]:
# Hover Tool

from bokeh.io import show
from bokeh.models import LinearInterpolator
from bokeh.plotting import figure
from bokeh.models import CategoricalColorMapper
from bokeh.palettes import Spectral6
from bokeh.models import HoverTool


hover = HoverTool(tooltips='@country')

p = figure(
    height = 400, 
    width = 600,
    x_axis_type = 'log',
    tools = [hover],
    x_range = (100,100000), y_range = (0,100),
    title = "Life Expectancy vs GDP of Nations")


size_mapper = LinearInterpolator(
    x = [data.population.min(), data.population.max()],
    y = [10,100]
)

# Mapping color to region
color_mapper = CategoricalColorMapper(
    factors=list(data.continent.unique()),
    palette=Spectral6
)


p.circle(x='gdpPercap', y='lifeExp',alpha=0.6,source=source,
         size={'field':'population', 'transform':size_mapper},
         color={'field':'continent', 'transform':color_mapper},
         legend='continent'
        )

p.xaxis.axis_label = "GDP per Capita"
p.yaxis.axis_label = "Life Expectancy"


from bokeh.models import NumeralTickFormatter
p.xaxis[0].formatter = NumeralTickFormatter(format='$0,')

# formating the legend
p.legend.location = (20, -6)
p.right.append(p.legend[0])
p.legend.border_line_color = None


show(p)